In [16]:
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import numpy as np
import random 

pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",100)

In [18]:
x = tf.constant([random.randint(1,100) for _ in range(100)],shape=[100,1])
print(x)

tf.Tensor(
[[36]
 [ 9]
 [86]
 [39]
 [42]
 [36]
 [ 5]
 [ 8]
 [10]
 [ 3]
 [46]
 [13]
 [11]
 [32]
 [55]
 [58]
 [16]
 [63]
 [62]
 [99]
 [48]
 [33]
 [85]
 [42]
 [29]
 [11]
 [83]
 [27]
 [30]
 [55]
 [12]
 [66]
 [62]
 [34]
 [97]
 [73]
 [ 5]
 [51]
 [35]
 [77]
 [15]
 [64]
 [34]
 [94]
 [55]
 [88]
 [38]
 [72]
 [ 6]
 [23]
 [67]
 [74]
 [67]
 [38]
 [40]
 [20]
 [29]
 [59]
 [74]
 [37]
 [51]
 [55]
 [99]
 [36]
 [79]
 [97]
 [57]
 [67]
 [23]
 [71]
 [11]
 [64]
 [78]
 [52]
 [89]
 [82]
 [44]
 [52]
 [10]
 [98]
 [96]
 [90]
 [65]
 [97]
 [22]
 [80]
 [24]
 [92]
 [72]
 [26]
 [ 9]
 [74]
 [24]
 [32]
 [40]
 [37]
 [83]
 [30]
 [86]
 [66]], shape=(100, 1), dtype=int32)


In [23]:
y = x * 2 + 1
y

<tf.Tensor: shape=(100, 1), dtype=int32, numpy=
array([[ 73],
       [ 19],
       [173],
       [ 79],
       [ 85],
       [ 73],
       [ 11],
       [ 17],
       [ 21],
       [  7],
       [ 93],
       [ 27],
       [ 23],
       [ 65],
       [111],
       [117],
       [ 33],
       [127],
       [125],
       [199],
       [ 97],
       [ 67],
       [171],
       [ 85],
       [ 59],
       [ 23],
       [167],
       [ 55],
       [ 61],
       [111],
       [ 25],
       [133],
       [125],
       [ 69],
       [195],
       [147],
       [ 11],
       [103],
       [ 71],
       [155],
       [ 31],
       [129],
       [ 69],
       [189],
       [111],
       [177],
       [ 77],
       [145],
       [ 13],
       [ 47],
       [135],
       [149],
       [135],
       [ 77],
       [ 81],
       [ 41],
       [ 59],
       [119],
       [149],
       [ 75],
       [103],
       [111],
       [199],
       [ 73],
       [159],
       [195],
       [115],
       [135],


In [93]:
from tensorflow.keras import regularizers,activations,initializers,constraints
from tensorflow.keras.layers import InputSpec
from tensorflow import dtypes
from tensorflow.keras import backend as K
class MyDenseLayer(keras.layers.Layer):
    
    def __init__(self,
                 units,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        # 将kwargs中的'input_dim'改为'input_shape'
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        
        super(MyDenseLayer,self).__init__(
            activity_regularizer=regularizers.get(activity_regularizer),**kwargs)
        
        # 输出的神经元个数
        self.units = int(units) if not isinstance(units,int) else units
        self.activation = activations.get(activation) # 激活函数
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer) # 权重矩阵初始化方法
        self.bias_initializer = initializers.get(bias_initializer) # 偏移矩阵初始化方法
        self.kernel_regularizer = regularizers.get(kernel_regularizer) # 权重矩阵正则化
        self.bias_regularizer = regularizers.get(bias_regularizer) # 偏移矩阵的正则化
        self.kernel_constraint = constraints.get(kernel_constraint) # 权重矩阵的约束条件：如非负constraints.non_neg
        self.bias_constraint = constraints.get(bias_constraint) # 偏移矩阵的约束条件
        
        self.supports_masking = True
        self.input_spec = InputSpec(min_ndim=2) 
    
    def build(self,input_shape):
        '''
        在__init__()方法和 call()方法之间被调用，在需要动态的使用输入的shape时需要重写：
        例如在全连接层时，我们需要使用上一层神经元的个数=input_shape[-1]
        '''
        dtype = dtypes.as_dtype(self.dtype or K.floatx())
        if not (dtype.is_floating or dtype.is_complex): # 全连接网络权重和偏移必须要float类型
            raise TypeError("全连接网络仅支持float类型；当前类型:%s"%(dtype,))
        
        last_dim = input_shape[-1]
        if last_dim is None:
            raise ValueError("输入层的最后一层的维度不能为None")
        
        self.input_spec = InputSpec(min_ndim=2,axes={-1:last_dim})
        # 定义权重矩阵
        self.kernel = self.add_weight(
            'kernel',
            shape=[last_dim,self.units], # 权重矩阵shape
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
            dtype=self.dtype,
            trainable=True)
        
        if self.use_bias: # 如果使用偏移
            self.bias = self.add_weight(
                'bias',
                shape=[self.units,],
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
                dtype=self.dtype,
                trainable=True)
        else:
            self.bias = None
        
        # 调用父类build方法，完成剩余操作，其中父类中最终会设置self.built = True
        super(MyDenseLayer,self).build(input_shape)
    
    def call(self,inputs,**kwargs):
        '''
        在build方法执行完之后，将会执行call方法
        '''
        rank = inputs.shape.rank
        if rank is not None and rank > 2: 
            # 如果input的维度大于2维，那么将会取inpus的最后一维与权重举证的第一维进行矩阵运算进行
            outputs = tf.tensordot(inputs,self.kernel,axes=[[rank-1],[0]])
            if not tf.executing_eagerly(): # 如果没有eager模型，执行下面shape的设置操作，如果
                shape = inputs.shape.as_list()
                output_shape = shape[:-1] + [self.units]
                outputs.set_shape(output_shape)
        else:
            inputs = tf.cast(inputs,self._compute_dtype) # 转换输入矩阵类型
            if K.is_sparse(inputs): # 如果是inputs 是离散的，可以在建立tensor时指定
                # 一般情况下都是连续变量
                outputs = tf.sparse.sparse_dense_matmul(inputs,self.kernel)
            else:
                outputs = tf.matmul(inputs,self.kernel) # 正常的矩阵乘法
#         outputs = tf.matmul(inputs,self.kernel) # 正常的矩阵乘法
        if self.use_bias:
            outputs = tf.nn.bias_add(outputs,self.bias)
        if self.activation is not None:
            return self.activation(outputs)
        return outputs
    
    def get_config(self):
        config = {
            'units':self.units,
            'activation':activations.serialize(self.activation),
            'user_bias':self.use_bias,
            'kernel_initializer':initializers.serialize(self.kernel_initializer),
            'bias_initializer':initializers.serialize(self.bias_initializer),
            'kernel_regularizer':regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer':regularizers.serialize(self.bias_regularizer),
            'kernel_constraint':constraints.serialize(self.kernel_constraint),
            'bias_constraint':constraints.serialize(self.bias_constraint),
        }
        base_config = super(MyDenseLayer,self).get_config()
        return dict(list(base_config.items())+list(config.items()))

In [94]:
input_layer = keras.layers.Input(shape=(1,),name="x")
out_put = MyDenseLayer(1)(input_layer)
# out_put = keras.layers.Dense(1)(input_layer)

model = tf.keras.models.Model(inputs=[input_layer],
                              outputs=out_put)
model.summary()
model.compile('adam','mse')

1


In [96]:
model.fit({'x':x},y,epochs=1000,batch_size=32)

Epoch 1/1000
4/4 [==============================] - 0s 1ms/step - loss: 12478.4971
Epoch 2/1000
4/4 [==============================] - 0s 1ms/step - loss: 12425.3174
Epoch 3/1000
4/4 [==============================] - 0s 1ms/step - loss: 12375.5059
Epoch 4/1000
4/4 [==============================] - 0s 946us/step - loss: 12323.9160
Epoch 5/1000
4/4 [==============================] - 0s 1ms/step - loss: 12272.0225
Epoch 6/1000
4/4 [==============================] - 0s 715us/step - loss: 12222.8242
Epoch 7/1000
4/4 [==============================] - 0s 665us/step - loss: 12171.2676
Epoch 8/1000
4/4 [==============================] - 0s 736us/step - loss: 12122.8633
Epoch 9/1000
4/4 [==============================] - 0s 734us/step - loss: 12070.7363
Epoch 10/1000
4/4 [==============================] - 0s 755us/step - loss: 12022.1191
Epoch 11/1000
4/4 [==============================] - 0s 760us/step - loss: 11970.8438
Epoch 12/1000
4/4 [==============================] - 0s 675us/step - lo

4/4 [==============================] - 0s 759us/step - loss: 8232.1201
Epoch 98/1000
4/4 [==============================] - 0s 814us/step - loss: 8191.8052
Epoch 99/1000
4/4 [==============================] - 0s 832us/step - loss: 8153.3369
Epoch 100/1000
4/4 [==============================] - 0s 914us/step - loss: 8114.9268
Epoch 101/1000
4/4 [==============================] - 0s 672us/step - loss: 8078.0674
Epoch 102/1000
4/4 [==============================] - 0s 716us/step - loss: 8042.7524
Epoch 103/1000
4/4 [==============================] - 0s 822us/step - loss: 8007.0088
Epoch 104/1000
4/4 [==============================] - 0s 913us/step - loss: 7971.1055
Epoch 105/1000
4/4 [==============================] - 0s 828us/step - loss: 7936.1011
Epoch 106/1000
4/4 [==============================] - 0s 707us/step - loss: 7900.8638
Epoch 107/1000
4/4 [==============================] - 0s 900us/step - loss: 7865.2798
Epoch 108/1000
4/4 [==============================] - 0s 892us/step - l

4/4 [==============================] - 0s 895us/step - loss: 5228.2046
Epoch 193/1000
4/4 [==============================] - 0s 796us/step - loss: 5202.8462
Epoch 194/1000
4/4 [==============================] - 0s 1ms/step - loss: 5176.5273
Epoch 195/1000
4/4 [==============================] - 0s 687us/step - loss: 5151.3511
Epoch 196/1000
4/4 [==============================] - 0s 754us/step - loss: 5124.7939
Epoch 197/1000
4/4 [==============================] - 0s 680us/step - loss: 5098.5659
Epoch 198/1000
4/4 [==============================] - 0s 799us/step - loss: 5071.7202
Epoch 199/1000
4/4 [==============================] - 0s 732us/step - loss: 5045.7627
Epoch 200/1000
4/4 [==============================] - 0s 790us/step - loss: 5018.6694
Epoch 201/1000
4/4 [==============================] - 0s 1ms/step - loss: 4994.3379
Epoch 202/1000
4/4 [==============================] - 0s 811us/step - loss: 4970.3315
Epoch 203/1000
4/4 [==============================] - 0s 921us/step - los

4/4 [==============================] - 0s 709us/step - loss: 3131.1787
Epoch 288/1000
4/4 [==============================] - 0s 746us/step - loss: 3114.7869
Epoch 289/1000
4/4 [==============================] - 0s 645us/step - loss: 3097.1548
Epoch 290/1000
4/4 [==============================] - 0s 791us/step - loss: 3080.5845
Epoch 291/1000
4/4 [==============================] - 0s 739us/step - loss: 3063.0938
Epoch 292/1000
4/4 [==============================] - 0s 865us/step - loss: 3045.3081
Epoch 293/1000
4/4 [==============================] - 0s 1ms/step - loss: 3028.9966
Epoch 294/1000
4/4 [==============================] - 0s 823us/step - loss: 3011.4424
Epoch 295/1000
4/4 [==============================] - 0s 886us/step - loss: 2994.3982
Epoch 296/1000
4/4 [==============================] - 0s 1ms/step - loss: 2977.1851
Epoch 297/1000
4/4 [==============================] - 0s 674us/step - loss: 2960.2498
Epoch 298/1000
4/4 [==============================] - 0s 691us/step - los

4/4 [==============================] - 0s 706us/step - loss: 1729.5719
Epoch 383/1000
4/4 [==============================] - 0s 641us/step - loss: 1718.2368
Epoch 384/1000
4/4 [==============================] - 0s 690us/step - loss: 1706.3356
Epoch 385/1000
4/4 [==============================] - 0s 686us/step - loss: 1694.7744
Epoch 386/1000
4/4 [==============================] - 0s 772us/step - loss: 1683.4376
Epoch 387/1000
4/4 [==============================] - 0s 764us/step - loss: 1671.8829
Epoch 388/1000
4/4 [==============================] - 0s 870us/step - loss: 1660.5203
Epoch 389/1000
4/4 [==============================] - 0s 791us/step - loss: 1649.6223
Epoch 390/1000
4/4 [==============================] - 0s 736us/step - loss: 1638.4591
Epoch 391/1000
4/4 [==============================] - 0s 677us/step - loss: 1627.7667
Epoch 392/1000
4/4 [==============================] - 0s 676us/step - loss: 1616.5707
Epoch 393/1000
4/4 [==============================] - 0s 723us/step -

...

4/4 [==============================] - 0s 815us/step - loss: 367.0708
Epoch 574/1000
4/4 [==============================] - 0s 788us/step - loss: 363.6700
Epoch 575/1000
4/4 [==============================] - 0s 1ms/step - loss: 360.4076
Epoch 576/1000
4/4 [==============================] - 0s 732us/step - loss: 357.1970
Epoch 577/1000
4/4 [==============================] - 0s 662us/step - loss: 354.0785
Epoch 578/1000
4/4 [==============================] - 0s 683us/step - loss: 350.7443
Epoch 579/1000
4/4 [==============================] - 0s 760us/step - loss: 347.5288
Epoch 580/1000
4/4 [==============================] - 0s 714us/step - loss: 344.1191
Epoch 581/1000
4/4 [==============================] - 0s 870us/step - loss: 340.7401
Epoch 582/1000
4/4 [==============================] - 0s 675us/step - loss: 337.1951
Epoch 583/1000
4/4 [==============================] - 0s 710us/step - loss: 333.8329
Epoch 584/1000
4/4 [==============================] - 0s 769us/step - loss: 330.74

4/4 [==============================] - 0s 866us/step - loss: 136.0098
Epoch 670/1000
4/4 [==============================] - 0s 728us/step - loss: 134.5366
Epoch 671/1000
4/4 [==============================] - 0s 764us/step - loss: 133.0282
Epoch 672/1000
4/4 [==============================] - 0s 899us/step - loss: 131.5849
Epoch 673/1000
4/4 [==============================] - 0s 794us/step - loss: 130.0445
Epoch 674/1000
4/4 [==============================] - 0s 764us/step - loss: 128.6633
Epoch 675/1000
4/4 [==============================] - 0s 1ms/step - loss: 127.2356
Epoch 676/1000
4/4 [==============================] - 0s 723us/step - loss: 125.7584
Epoch 677/1000
4/4 [==============================] - 0s 753us/step - loss: 124.2978
Epoch 678/1000
4/4 [==============================] - 0s 750us/step - loss: 122.8264
Epoch 679/1000
4/4 [==============================] - 0s 873us/step - loss: 121.3599
Epoch 680/1000
4/4 [==============================] - 0s 795us/step - loss: 119.89

4/4 [==============================] - 0s 723us/step - loss: 39.9983
Epoch 767/1000
4/4 [==============================] - 0s 652us/step - loss: 39.4374
Epoch 768/1000
4/4 [==============================] - 0s 827us/step - loss: 38.8859
Epoch 769/1000
4/4 [==============================] - 0s 779us/step - loss: 38.3407
Epoch 770/1000
4/4 [==============================] - 0s 807us/step - loss: 37.7994
Epoch 771/1000
4/4 [==============================] - 0s 1ms/step - loss: 37.2798
Epoch 772/1000
4/4 [==============================] - 0s 695us/step - loss: 36.7616
Epoch 773/1000
4/4 [==============================] - 0s 689us/step - loss: 36.2534
Epoch 774/1000
4/4 [==============================] - 0s 713us/step - loss: 35.7361
Epoch 775/1000
4/4 [==============================] - 0s 782us/step - loss: 35.2462
Epoch 776/1000
4/4 [==============================] - 0s 769us/step - loss: 34.7707
Epoch 777/1000
4/4 [==============================] - 0s 939us/step - loss: 34.2927
Epoch 778

4/4 [==============================] - 0s 709us/step - loss: 9.5514
Epoch 864/1000
4/4 [==============================] - 0s 787us/step - loss: 9.3983
Epoch 865/1000
4/4 [==============================] - 0s 711us/step - loss: 9.2331
Epoch 866/1000
4/4 [==============================] - 0s 808us/step - loss: 9.0889
Epoch 867/1000
4/4 [==============================] - 0s 881us/step - loss: 8.9332
Epoch 868/1000
4/4 [==============================] - 0s 827us/step - loss: 8.7788
Epoch 869/1000
4/4 [==============================] - 0s 1ms/step - loss: 8.6314
Epoch 870/1000
4/4 [==============================] - 0s 805us/step - loss: 8.4934
Epoch 871/1000
4/4 [==============================] - 0s 650us/step - loss: 8.3435
Epoch 872/1000
4/4 [==============================] - 0s 945us/step - loss: 8.2047
Epoch 873/1000
4/4 [==============================] - 0s 625us/step - loss: 8.0638
Epoch 874/1000
4/4 [==============================] - 0s 724us/step - loss: 7.9275
Epoch 875/1000
4/4 [=

Epoch 962/1000
4/4 [==============================] - 0s 946us/step - loss: 1.6671
Epoch 963/1000
4/4 [==============================] - 0s 785us/step - loss: 1.6370
Epoch 964/1000
4/4 [==============================] - 0s 695us/step - loss: 1.6063
Epoch 965/1000
4/4 [==============================] - 0s 680us/step - loss: 1.5760
Epoch 966/1000
4/4 [==============================] - 0s 744us/step - loss: 1.5474
Epoch 967/1000
4/4 [==============================] - 0s 700us/step - loss: 1.5187
Epoch 968/1000
4/4 [==============================] - 0s 793us/step - loss: 1.4891
Epoch 969/1000
4/4 [==============================] - 0s 699us/step - loss: 1.4623
Epoch 970/1000
4/4 [==============================] - 0s 847us/step - loss: 1.4362
Epoch 971/1000
4/4 [==============================] - 0s 884us/step - loss: 1.4120
Epoch 972/1000
4/4 [==============================] - 0s 851us/step - loss: 1.3852
Epoch 973/1000
4/4 [==============================] - 0s 1ms/step - loss: 1.3613
Epoch 

In [97]:
model.predict({'x':np.array([[5]])})

array([[11.776194]], dtype=float32)